In [4]:
# Credits: https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# LSTM for sequence classification in the IMDB dataset

### Import libraries

In [6]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)

### Split Train and Test data

In [11]:
# https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification

# load the dataset but only keep the top n words, zero the rest
top_words=5000 #means if any word with index<5000 will be not counted
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [13]:
print(X_train[1]) #first datapoint of the X_train
print(type(X_train[1]))
print(len(X_train[1]))

[1, 194, 1153, 194, 2, 78, 228, 5, 6, 1463, 4369, 2, 134, 26, 4, 715, 8, 118, 1634, 14, 394, 20, 13, 119, 954, 189, 102, 5, 207, 110, 3103, 21, 14, 69, 188, 8, 30, 23, 7, 4, 249, 126, 93, 4, 114, 9, 2300, 1523, 5, 647, 4, 116, 9, 35, 2, 4, 229, 9, 340, 1322, 4, 118, 9, 4, 130, 4901, 19, 4, 1002, 5, 89, 29, 952, 46, 37, 4, 455, 9, 45, 43, 38, 1543, 1905, 398, 4, 1649, 26, 2, 5, 163, 11, 3215, 2, 4, 1153, 9, 194, 775, 7, 2, 2, 349, 2637, 148, 605, 2, 2, 15, 123, 125, 68, 2, 2, 15, 349, 165, 4362, 98, 5, 4, 228, 9, 43, 2, 1157, 15, 299, 120, 5, 120, 174, 11, 220, 175, 136, 50, 9, 4373, 228, 2, 5, 2, 656, 245, 2350, 5, 4, 2, 131, 152, 491, 18, 2, 32, 2, 1212, 14, 9, 6, 371, 78, 22, 625, 64, 1382, 9, 8, 168, 145, 23, 4, 1690, 15, 16, 4, 1355, 5, 28, 6, 52, 154, 462, 33, 89, 78, 285, 16, 145, 95]
<class 'list'>
189


### Truncate/Pad input sequences

Padding means appending with zeros if the length of the word is less than 600.                                        
and stripping if the length of the word is more than 600                                                              
Why Padding? -> We can do without padding also RNN allows but in that case SGD we have to use with one sequence at a time. If we make all words with same length then we can use SGD with some batch size and wchich is fast.

In [16]:
max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length) #this method does pre-truncating zeros to the words
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [20]:
# below code to just confirm after trancating for one datapoint
print(X_train.shape)
print("----------------------------------------------------------------------------------")
print(X_train[1])

(25000, 600)
----------------------------------------------------------------------------------
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0  

### Network Structure

In [34]:
# create the model
#Reference: https://datascience.stackexchange.com/questions/10615/number-of-parameters-in-an-lstm-model
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vector_length, input_length=max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 600, 32)           160000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=64)

C:\Users\shashi\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.
C:\Users\shashi\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
25000/25000 [==============================] - 223s 9ms/step - loss: 0.4544 - accuracy: 0.7784
Epoch 2/10
25000/25000 [==============================] - 227s 9ms/step - loss: 0.2992 - accuracy: 0.8802
Epoch 3/10
25000/25000 [==============================] - 228s 9ms/step - loss: 0.2723 - accuracy: 0.8949
Epoch 4/10
25000/25000 [==============================] - 232s 9ms/step - loss: 0.2375 - accuracy: 0.9118
Epoch 5/10
25000/25000 [==============================] - 230s 9ms/step - loss: 0.2187 - accuracy: 0.9176
Epoch 6/10
25000/25000 [==============================] - 230s 9ms/step - loss: 0.1990 - accuracy: 0.9255
Epoch 7/10
25000/25000 [==============================] - 235s 9ms/step - loss: 0.1793 - accuracy: 0.9329
Epoch 8/10
25000/25000 [==============================] - 239s 10ms/step - loss: 0.1696 - accuracy: 0.9383
Epoch 9/10
25000/25000 [==============================] - 239s 10ms/step - loss: 0.1371 - accuracy: 0.9494
Epoch 10/10
25000/25000 [===================

In [36]:
#Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy :", (scores[1]*100))

Accuracy : 85.93999743461609


5000